- 데이터셋:
    * 총 운항 기록 24/07/01~ 24/12/31
    * 총 이상운항 기록 24/07/01 ~ 24/12/31
- 가설 : 기상/기술 조건에 의한 항공지연이 발생하면, 추가적인 접속/연결 지연 발생률이 증가할 것이다.
- 가설 : 기술적 문제로 인한 지연은 다른 항공편에 미치는 영향이 크며, 연쇄 지연의 발생 확률이 기상 조건에 의한 지연보다 높다.
검증 목표 : 총 운항기록중 시간대별로 여객기와 화물수송기를 분류, 최초지연과 그 이후 이어지는 지연사건이 몇건이나 되는지 확인하여, 지연사고 최초발생과 그 이후 생기는 연결/접속 지연사고의 관계성을 밝힘



In [140]:
## [1] 모듈 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

import utils

In [141]:
## 데이터 로딩및 확인
dataDF01 = pd.read_excel('./DATA/normal/항공기출도착현황240101.xlsx')
dataDF02 = pd.read_excel('./DATA/normal/항공기출도착현황240201.xlsx')
dataDF03 = pd.read_excel('./DATA/normal/항공기출도착현황240301.xlsx')
dataDF04 = pd.read_excel('./DATA/normal/항공기출도착현황240401.xlsx')
dataDF05 = pd.read_excel('./DATA/normal/항공기출도착현황240501.xlsx')
dataDF06 = pd.read_excel('./DATA/normal/항공기출도착현황240601.xlsx')
dataDF07 = pd.read_excel('./DATA/normal/항공기출도착현황240701.xlsx')
dataDF08 = pd.read_excel('./DATA/normal/항공기출도착현황240801.xlsx')
dataDF09 = pd.read_excel('./DATA/normal/항공기출도착현황240901.xlsx')
dataDF10 = pd.read_excel('./DATA/normal/항공기출도착현황241001.xlsx')
dataDF11 = pd.read_excel('./DATA/normal/항공기출도착현황241101.xlsx')
dataDF12 = pd.read_excel('./DATA/normal/항공기출도착현황241201.xlsx')

# 계획시간 별로 소트하여 지연 현황을 찾고, 해당 지연 원인을 abnormal 폴더에서 해당날짜와 같은 날짜에서 찾아오기


c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\styleshee

In [142]:
dataDF07.tail(20)

,출/도착구분,날짜,항공사,편명,출발공항코드,출발공항명,도착공항코드,도착공항명,계획시간,예상시간,도착시간,구분,현황
18479,도착,20240731,대한항공,KE782,FUK,후쿠오카,ICN,인천,22:35,22:48,22:40,여객,도착
18480,도착,20240731,산동항공,SC8003,TNA,지난,ICN,인천,22:40,22:34,22:25,여객,도착
18481,도착,20240731,대한항공,KE654,BKK,수안나폼(방콕),ICN,인천,22:40,23:09,22:59,여객,지연
18482,도착,20240731,산동항공,SC4617,TAO,청도,ICN,인천,22:45,22:43,22:35,여객,도착
18483,도착,20240731,에어마카오,NX826,MFM,마카오,ICN,인천,22:45,22:41,22:31,여객,도착
18484,도착,20240731,비엣젯항공,VJ974,PQC,푸꾸옥,ICN,인천,22:45,23:00,22:52,여객,지연
18485,도착,20240731,비엣젯항공,VJ838,CXR,깜라인(나트랑),ICN,인천,22:45,23:03,22:59,여객,지연
18486,도착,20240731,대한항공,KE176,HKG,홍콩,ICN,인천,22:45,23:04,22:55,여객,지연
18487,도착,20240731,대한항공,KE8284,ORD,시카고,ICN,인천,22:50,22:16,22:02,화물,도착
18488,도착,20240731,에어서울,RS704,NRT,도쿄/나리타,ICN,인천,22:50,23:33,23:24,여객,지연


In [143]:
dataDF07.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18499 entries, 0 to 18498
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   출/도착구분  18499 non-null  object
 1   날짜      18499 non-null  int64 
 2   항공사     18498 non-null  object
 3   편명      18499 non-null  object
 4   출발공항코드  18499 non-null  object
 5   출발공항명   18499 non-null  object
 6   도착공항코드  18499 non-null  object
 7   도착공항명   18499 non-null  object
 8   계획시간    18499 non-null  object
 9   예상시간    18499 non-null  object
 10  도착시간    18499 non-null  object
 11  구분      18499 non-null  object
 12  현황      18498 non-null  object
dtypes: int64(1), object(12)
memory usage: 1.8+ MB


In [144]:
# 데이터 분류방법 / 조건문  / groupby
dataDF01 = dataDF01.sort_values(['날짜', '계획시간'])
dataDF02 = dataDF02.sort_values(['날짜', '계획시간'])
dataDF03 = dataDF03.sort_values(['날짜', '계획시간'])
dataDF04 = dataDF04.sort_values(['날짜', '계획시간'])
dataDF05 = dataDF05.sort_values(['날짜', '계획시간'])
dataDF06 = dataDF06.sort_values(['날짜', '계획시간'])
dataDF07 = dataDF07.sort_values(['날짜', '계획시간'])
dataDF08 = dataDF08.sort_values(['날짜', '계획시간'])
dataDF09 = dataDF09.sort_values(['날짜', '계획시간'])
dataDF10 = dataDF10.sort_values(['날짜', '계획시간'])
dataDF11 = dataDF11.sort_values(['날짜', '계획시간'])
dataDF12 = dataDF12.sort_values(['날짜', '계획시간'])
dataDF07.head()
# dataDF07.loc[(dataDF07['현황']=='지연')]
dataDF07.loc[:,'현황']
(dataDF07['현황'] == '지연').idxmax()


0

In [145]:
dataDF07['현황'] == '지연'

0         True
1         True
2         True
3        False
4        False
         ...  
18494    False
18495    False
18496    False
18497    False
18498    False
Name: 현황, Length: 18499, dtype: bool

In [146]:
""" def findlate(df, end_idx):
    # if end_idx == len(df.index):
    
    first_idx = (df.loc[end_idx:,'현황'] == '지연').idxmax()
    first_not_idx = (df.loc[first_idx:,'현황'] != '지연').idxmax()
    print('first_idx = ', first_idx)
    print('first_not_idx = ', first_not_idx)
    
    lateCount = 0 
    
    for n in np.arange(first_idx, first_not_idx):
        lateDate = df.loc[n,'날짜']
        lateName = df.loc[n,'편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        # print(lateReason, lateReason.dtype)
        if lateReason.str.contains('연결').any():         
            lateCount += 1
   
    if lateDate:    
        if (lateCount, lateDate) in lateDF.index and lateDate in lateDF.columns:
            lateDF.loc[lateCount, lateDate] += 1
        else:
            lateDF.loc[lateCount, lateDate] = 1
    
    # print('end_idx = ', first_not_idx)
    if first_not_idx >= len(dataDF07.index):
        print( lateDF )
        quit
    
    return findlate(df, first_not_idx)
         """

' def findlate(df, end_idx):\n    # if end_idx == len(df.index):\n    \n    first_idx = (df.loc[end_idx:,\'현황\'] == \'지연\').idxmax()\n    first_not_idx = (df.loc[first_idx:,\'현황\'] != \'지연\').idxmax()\n    print(\'first_idx = \', first_idx)\n    print(\'first_not_idx = \', first_not_idx)\n    \n    lateCount = 0 \n    \n    for n in np.arange(first_idx, first_not_idx):\n        lateDate = df.loc[n,\'날짜\']\n        lateName = df.loc[n,\'편명\']\n        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")\n        lateReason = lateDay.loc[(lateDay[\'항공편명\'] == lateName), \'비정상원인\']\n        # print(lateReason, lateReason.dtype)\n        if lateReason.str.contains(\'연결\').any():         \n            lateCount += 1\n   \n    if lateDate:    \n        if (lateCount, lateDate) in lateDF.index and lateDate in lateDF.columns:\n            lateDF.loc[lateCount, lateDate] += 1\n        else:\n            lateDF.loc[lateCount, lateDate] = 1\n    \n    # print(\'end_idx = \', fir

In [147]:
def findlate(df, start_idx):
    end_idx = len(df.index)
    lateDF = pd.DataFrame(index=range(30))
   
    while start_idx < end_idx:
        first_idx = (df.loc[start_idx:, '현황'] == '지연').idxmax()
        lateDate = df.loc[first_idx, '날짜']
        lateName = df.loc[first_idx, '편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        if not lateReason.str.contains('항공기 연결').any(): 
            first_not_idx = (df.loc[first_idx:, '현황'] != '지연').idxmax()
            
            # print('first_idx = ', first_idx)
            # print('first_not_idx = ', first_not_idx)

            lateCount = 0 

            for n in np.arange(first_idx, first_not_idx):
                lateDate = df.loc[n, '날짜']
                lateName = df.loc[n, '편명']
                lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
                lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
                
                if lateReason.str.contains('연결').any() | lateReason.str.contains('접속').any():         
                    lateCount += 1

            if lateDate in lateDF.columns:  
                # print(lateCount, lateDate,'b')
                lateDF.loc[lateCount, lateDate] += 1
            else:
                # print(lateCount, lateDate, 'a')
                lateDF[lateDate] = [0] * len(lateDF)
                lateDF.loc[lateCount, lateDate] = 1
            
            if first_idx >= first_not_idx:
                txtname = str(lateDate)[4:6]
                globals()[f"lateDF{txtname}"] = lateDF.copy()
                print(f"lateDF{txtname} 생성")
                break  

            start_idx = first_not_idx  
        else:
            start_idx = first_idx+1
    
    txtname = str(lateDate)[4:6]
    globals()[f"lateDF{txtname}"] = lateDF.copy()
    print(f"lateDF{txtname} 생성")

In [148]:
findlate(dataDF01,0)
findlate(dataDF02,0)
findlate(dataDF03,0)
findlate(dataDF04,0)
findlate(dataDF05,0)
findlate(dataDF06,0)
findlate(dataDF07,0)
findlate(dataDF08,0)
findlate(dataDF09,0)
findlate(dataDF10,0)
findlate(dataDF11,0)
findlate(dataDF12,0)

lateDF01 생성
lateDF01 생성
lateDF02 생성
lateDF02 생성
lateDF03 생성
lateDF03 생성
lateDF04 생성
lateDF04 생성
lateDF05 생성
lateDF05 생성
lateDF06 생성
lateDF06 생성
lateDF07 생성
lateDF07 생성
lateDF08 생성
lateDF08 생성
lateDF09 생성
lateDF09 생성
lateDF10 생성
lateDF10 생성
lateDF11 생성
lateDF11 생성
lateDF12 생성
lateDF12 생성


In [149]:
lateDF24sh = pd.DataFrame({'1월 합계':lateDF01.sum(axis=1),
                          '2월 합계':lateDF02.sum(axis=1),
                          '3월 합계':lateDF03.sum(axis=1),
                          '4월 합계':lateDF04.sum(axis=1),
                          '5월 합계':lateDF05.sum(axis=1),
                          '6월 합계':lateDF06.sum(axis=1),
                          '7월 합계':lateDF07.sum(axis=1),
                          '8월 합계':lateDF08.sum(axis=1),
                          '9월 합계':lateDF09.sum(axis=1),
                          '10월 합계':lateDF10.sum(axis=1),
                          '11월 합계':lateDF11.sum(axis=1),
                          '12월 합계':lateDF12.sum(axis=1)})
lateDF24sh.head(20)

,1월 합계,2월 합계,3월 합계,4월 합계,5월 합계,6월 합계,7월 합계,8월 합계,9월 합계,10월 합계,11월 합계,12월 합계
0,1876,1741,1787,1578,1668,1908,2196,2130,1897,1871,1909,2097
1,632,642,553,581,629,732,1050,920,822,811,726,962
2,95,89,57,70,81,128,258,180,120,147,106,150
3,19,25,16,6,15,29,65,42,31,25,42,35
4,3,5,0,0,0,6,30,13,10,7,19,5
5,1,1,0,1,0,8,11,4,4,0,2,0
6,0,2,0,0,0,3,3,1,1,0,4,0
7,0,0,1,0,0,2,1,1,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,0,0


In [150]:
lateDF24sh.to_csv('./DATA/result/lateDF24sh.csv')

In [151]:
def findlate_weather(df, start_idx):
    end_idx = len(df.index)
    lateDF = pd.DataFrame(index=range(30))
    
    # print(len(df.index))
    while start_idx < end_idx:
        # print(start_idx, end_idx)
        first_idx = (df.loc[start_idx:, '현황'] == '지연').idxmax()
        lateDate = df.loc[first_idx, '날짜']
        lateName = df.loc[first_idx, '편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        if lateReason.str.contains('기상').any(): 
            first_not_idx = (df.loc[first_idx:, '현황'] != '지연').idxmax()
            
            # print('first_idx = ', first_idx)
            # print('first_not_idx = ', first_not_idx)

            lateCount = 0 

            for n in np.arange(first_idx, first_not_idx):
                lateDate = df.loc[n, '날짜']
                lateName = df.loc[n, '편명']
                lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
                lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
                
                if lateReason.str.contains('연결').any() | lateReason.str.contains('접속').any():         
                    lateCount += 1

            if lateDate in lateDF.columns:  # lateDate 변수가 정의되었는지 확인
                # print(lateCount, lateDate,'b')
                lateDF.loc[lateCount, lateDate] += 1
            else:
                
                lateDF[lateDate] = [0] * len(lateDF)
                lateDF.loc[lateCount, lateDate] = 1
            
            if first_idx >= first_not_idx or first_not_idx == len(df.index):
                # print(lateCount, lateDate, 'a')
                txtname = str(lateDate)[4:6]
                globals()[f"lateDFwea{txtname}"] = lateDF.copy()
                print(f"lateDFwea{txtname} 생성")
                break  # 재귀 대신 종료

            start_idx = first_not_idx  # 다음 반복을 위해 업데이트
        else:
            start_idx = first_idx+1
            
            
    txtname = str(lateDate)[4:6]
    globals()[f"lateDFwea{txtname}"] = lateDF.copy()
    print(f"lateDFwea{txtname} 생성")

In [152]:
findlate_weather(dataDF01,0)
findlate_weather(dataDF02,0)
findlate_weather(dataDF03,0)
findlate_weather(dataDF04,0)
findlate_weather(dataDF05,0)
findlate_weather(dataDF06,0)
findlate_weather(dataDF07,0)
findlate_weather(dataDF08,0)
findlate_weather(dataDF09,0)
findlate_weather(dataDF10,0)
findlate_weather(dataDF11,0)
findlate_weather(dataDF12,0)

lateDFwea01 생성
lateDFwea02 생성
lateDFwea03 생성
lateDFwea04 생성
lateDFwea05 생성
lateDFwea06 생성
lateDFwea07 생성
lateDFwea08 생성
lateDFwea09 생성
lateDFwea10 생성
lateDFwea11 생성
lateDFwea12 생성


In [153]:
lateDF24shwea = pd.DataFrame({'1월 합계':lateDFwea01.sum(axis=1),
                          '2월 합계':lateDFwea02.sum(axis=1),
                          '3월 합계':lateDFwea03.sum(axis=1),
                          '4월 합계':lateDFwea04.sum(axis=1),
                          '5월 합계':lateDFwea05.sum(axis=1),
                          '6월 합계':lateDFwea06.sum(axis=1),
                          '7월 합계':lateDFwea07.sum(axis=1),
                          '8월 합계':lateDFwea08.sum(axis=1),
                          '9월 합계':lateDFwea09.sum(axis=1),
                          '10월 합계':lateDFwea10.sum(axis=1),
                          '11월 합계':lateDFwea11.sum(axis=1),
                          '12월 합계':lateDFwea12.sum(axis=1)})

In [154]:
lateDF24shwea
lateDF24shwea.to_csv('./DATA/result/lateDF24shwea.csv')

In [155]:
def findlate_ATFM(df, start_idx):
    end_idx = len(df.index)
    lateDF = pd.DataFrame(index=range(30))
    
    # print(len(df.index))
    while start_idx < end_idx:
        # print(start_idx, end_idx)
        first_idx = (df.loc[start_idx:, '현황'] == '지연').idxmax()
        lateDate = df.loc[first_idx, '날짜']
        lateName = df.loc[first_idx, '편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        if lateReason.str.contains('ATFM|운항통제|공항시설|공항, 관제',).any(): 
            first_not_idx = (df.loc[first_idx:, '현황'] != '지연').idxmax()
            
            # print('first_idx = ', first_idx)
            # print('first_not_idx = ', first_not_idx)

            lateCount = 0 

            for n in np.arange(first_idx, first_not_idx):
                lateDate = df.loc[n, '날짜']
                lateName = df.loc[n, '편명']
                lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
                lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
                
                if lateReason.str.contains('연결').any() | lateReason.str.contains('접속').any():         
                    lateCount += 1

            if lateDate in lateDF.columns:  # lateDate 변수가 정의되었는지 확인
                # print(lateCount, lateDate,'b')
                lateDF.loc[lateCount, lateDate] += 1
            else:
                
                lateDF[lateDate] = [0] * len(lateDF)
                lateDF.loc[lateCount, lateDate] = 1
            
            if first_idx >= first_not_idx or first_not_idx == len(df.index):
                # print(lateCount, lateDate, 'a')
                txtname = str(lateDate)[4:6]
                globals()[f"lateDFatfm{txtname}"] = lateDF.copy()
                print(f"lateDFatfm{txtname} 생성")
                break  # 재귀 대신 종료

            start_idx = first_not_idx  # 다음 반복을 위해 업데이트
        else:
            start_idx = first_idx+1
            
            
    txtname = str(lateDate)[4:6]
    globals()[f"lateDFatfm{txtname}"] = lateDF.copy()
    print(f"lateDFatfm{txtname} 생성")

In [156]:
findlate_ATFM(dataDF01,0)
findlate_ATFM(dataDF02,0)
findlate_ATFM(dataDF03,0)
findlate_ATFM(dataDF04,0)
findlate_ATFM(dataDF05,0)
findlate_ATFM(dataDF06,0)
findlate_ATFM(dataDF07,0)
findlate_ATFM(dataDF08,0)
findlate_ATFM(dataDF09,0)
findlate_ATFM(dataDF10,0)
findlate_ATFM(dataDF11,0)
findlate_ATFM(dataDF12,0)

lateDFatfm01 생성
lateDFatfm02 생성
lateDFatfm03 생성
lateDFatfm04 생성
lateDFatfm05 생성
lateDFatfm06 생성
lateDFatfm07 생성
lateDFatfm08 생성
lateDFatfm09 생성
lateDFatfm10 생성
lateDFatfm11 생성
lateDFatfm12 생성


In [157]:
lateDF24shATFM = pd.DataFrame({'1월 합계':lateDFatfm01.sum(axis=1),
                          '2월 합계':lateDFatfm02.sum(axis=1),
                          '3월 합계':lateDFatfm03.sum(axis=1),
                          '4월 합계':lateDFatfm04.sum(axis=1),
                          '5월 합계':lateDFatfm05.sum(axis=1),
                          '6월 합계':lateDFatfm06.sum(axis=1),
                          '7월 합계':lateDFatfm07.sum(axis=1),
                          '8월 합계':lateDFatfm08.sum(axis=1),
                          '9월 합계':lateDFatfm09.sum(axis=1),
                          '10월 합계':lateDFatfm10.sum(axis=1),
                          '11월 합계':lateDFatfm11.sum(axis=1),
                          '12월 합계':lateDFatfm12.sum(axis=1)})

In [158]:
lateDF24shATFM
lateDF24shATFM.to_csv('./DATA/result/lateDF24shATFM.csv')

In [159]:
def findlate_pass(df, start_idx):
    end_idx = len(df.index)
    lateDF = pd.DataFrame(index=range(30))
    
    # print(len(df.index))
    while start_idx < end_idx:
        # print(start_idx, end_idx)
        first_idx = (df.loc[start_idx:, '현황'] == '지연').idxmax()
        lateDate = df.loc[first_idx, '날짜']
        lateName = df.loc[first_idx, '편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        if lateReason.str.contains('승객|수화물',).any(): 
            first_not_idx = (df.loc[first_idx:, '현황'] != '지연').idxmax()
            
            # print('first_idx = ', first_idx)
            # print('first_not_idx = ', first_not_idx)

            lateCount = 0 

            for n in np.arange(first_idx, first_not_idx):
                lateDate = df.loc[n, '날짜']
                lateName = df.loc[n, '편명']
                lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
                lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
                
                if lateReason.str.contains('연결').any() | lateReason.str.contains('접속').any():         
                    lateCount += 1

            if lateDate in lateDF.columns:  # lateDate 변수가 정의되었는지 확인
                # print(lateCount, lateDate,'b')
                lateDF.loc[lateCount, lateDate] += 1
            else:
                
                lateDF[lateDate] = [0] * len(lateDF)
                lateDF.loc[lateCount, lateDate] = 1
            
            if first_idx >= first_not_idx or first_not_idx == len(df.index):
                # print(lateCount, lateDate, 'a')
                txtname = str(lateDate)[4:6]
                globals()[f"lateDFpass{txtname}"] = lateDF.copy()
                print(f"lateDFpass{txtname} 생성")
                break  # 재귀 대신 종료

            start_idx = first_not_idx  # 다음 반복을 위해 업데이트
        else:
            start_idx = first_idx+1
            
            
    txtname = str(lateDate)[4:6]
    globals()[f"lateDFpass{txtname}"] = lateDF.copy()
    print(f"lateDFpass{txtname} 생성")

In [160]:
findlate_pass(dataDF01,0)
findlate_pass(dataDF02,0)
findlate_pass(dataDF03,0)
findlate_pass(dataDF04,0)
findlate_pass(dataDF05,0)
findlate_pass(dataDF06,0)
findlate_pass(dataDF07,0)
findlate_pass(dataDF08,0)
findlate_pass(dataDF09,0)
findlate_pass(dataDF10,0)
findlate_pass(dataDF11,0)
findlate_pass(dataDF12,0)

lateDFpass01 생성
lateDFpass02 생성
lateDFpass03 생성
lateDFpass04 생성
lateDFpass05 생성
lateDFpass06 생성
lateDFpass07 생성
lateDFpass08 생성
lateDFpass09 생성
lateDFpass10 생성
lateDFpass11 생성
lateDFpass12 생성


In [161]:
lateDF24shpass = pd.DataFrame({'1월 합계':lateDFpass01.sum(axis=1),
                          '2월 합계':lateDFpass02.sum(axis=1),
                          '3월 합계':lateDFpass03.sum(axis=1),
                          '4월 합계':lateDFpass04.sum(axis=1),
                          '5월 합계':lateDFpass05.sum(axis=1),
                          '6월 합계':lateDFpass06.sum(axis=1),
                          '7월 합계':lateDFpass07.sum(axis=1),
                          '8월 합계':lateDFpass08.sum(axis=1),
                          '9월 합계':lateDFpass09.sum(axis=1),
                          '10월 합계':lateDFpass10.sum(axis=1),
                          '11월 합계':lateDFpass11.sum(axis=1),
                          '12월 합계':lateDFpass12.sum(axis=1)})

In [162]:
lateDF24shpass.to_csv('./DATA/result/lateDF24shpass.csv')

In [163]:
def findcanc_weather(df, start_idx):
    end_idx = len(df.index)
    lateDF = pd.DataFrame(index=range(30))
    
    # print(len(df.index))
    while start_idx < end_idx:
        # print(start_idx, end_idx)
        first_idx = (df.loc[start_idx:, '현황'] == '취소').idxmax()
        lateDate = df.loc[first_idx, '날짜']
        lateName = df.loc[first_idx, '편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        if lateReason.str.contains('기상').any(): 
            first_not_idx = (df.loc[first_idx:, '현황'] != '취소').idxmax()
            
            # print('first_idx = ', first_idx)
            # print('first_not_idx = ', first_not_idx)

            lateCount = 0 

            for n in np.arange(first_idx, first_not_idx):
                lateDate = df.loc[n, '날짜']
                lateName = df.loc[n, '편명']
                lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
                lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
                
                if lateReason.str.contains('연결').any() | lateReason.str.contains('접속').any():         
                    lateCount += 1

            if lateDate in lateDF.columns:  # lateDate 변수가 정의되었는지 확인
                # print(lateCount, lateDate,'b')
                lateDF.loc[lateCount, lateDate] += 1
            else:
                
                lateDF[lateDate] = [0] * len(lateDF)
                lateDF.loc[lateCount, lateDate] = 1
            
            if first_idx >= first_not_idx or first_not_idx == len(df.index):
                # print(lateCount, lateDate, 'a')
                txtname = str(lateDate)[4:6]
                globals()[f"cancDFwea{txtname}"] = lateDF.copy()
                print(f"cancDFwea{txtname} 생성")
                break  # 재귀 대신 종료

            start_idx = first_not_idx  # 다음 반복을 위해 업데이트
        else:
            start_idx = first_idx+1
            
            
    txtname = str(lateDate)[4:6]
    globals()[f"cancDFwea{txtname}"] = lateDF.copy()
    print(f"cancDFwea{txtname} 생성")

In [164]:
findcanc_weather(dataDF01,0)
findcanc_weather(dataDF02,0)
findcanc_weather(dataDF03,0)
findcanc_weather(dataDF04,0)
findcanc_weather(dataDF05,0)
findcanc_weather(dataDF06,0)
findcanc_weather(dataDF07,0)
findcanc_weather(dataDF08,0)
findcanc_weather(dataDF09,0)
findcanc_weather(dataDF10,0)
findcanc_weather(dataDF11,0)
findcanc_weather(dataDF12,0)

cancDFwea01 생성
cancDFwea01 생성
cancDFwea02 생성
cancDFwea03 생성
cancDFwea04 생성
cancDFwea05 생성
cancDFwea05 생성
cancDFwea06 생성
cancDFwea07 생성
cancDFwea08 생성
cancDFwea09 생성
cancDFwea10 생성
cancDFwea11 생성
cancDFwea11 생성
cancDFwea12 생성


In [165]:
cancDF24shwea = pd.DataFrame({'1월 합계':cancDFwea01.sum(axis=1),
                          '2월 합계':cancDFwea02.sum(axis=1),
                          '3월 합계':cancDFwea03.sum(axis=1),
                          '4월 합계':cancDFwea04.sum(axis=1),
                          '5월 합계':cancDFwea05.sum(axis=1),
                          '6월 합계':cancDFwea06.sum(axis=1),
                          '7월 합계':cancDFwea07.sum(axis=1),
                          '8월 합계':cancDFwea08.sum(axis=1),
                          '9월 합계':cancDFwea09.sum(axis=1),
                          '10월 합계':cancDFwea10.sum(axis=1),
                          '11월 합계':cancDFwea11.sum(axis=1),
                          '12월 합계':cancDFwea12.sum(axis=1)})

In [166]:
cancDF24shwea.to_csv('./DATA/result/cancDF24shwea.csv')

In [167]:
totalLateDF = pd.DataFrame()
def total_late(df):
    df.loc[df['현황']=='지연',['현황']].count()
    print(df.loc[df['현황']=='지연',['현황']].count())
    

In [168]:
dataDF01.shape

(17572, 13)

In [169]:
dataDF01.loc[dataDF01['현황']=='지연',['현황']]

,현황
19,지연
23,지연
32,지연
33,지연
44,지연
...,...
17527,지연
17532,지연
17536,지연
17543,지연


In [170]:
total_late(dataDF01)
totalLateDF['1월'] = dataDF01.loc[dataDF01['현황']=='지연',['현황']].count()
total_late(dataDF02)
totalLateDF['2월'] = dataDF02.loc[dataDF02['현황']=='지연',['현황']].count()
total_late(dataDF03)
totalLateDF['3월'] = dataDF03.loc[dataDF03['현황']=='지연',['현황']].count()
total_late(dataDF04)
totalLateDF['4월'] = dataDF04.loc[dataDF04['현황']=='지연',['현황']].count()
total_late(dataDF05)
totalLateDF['5월'] = dataDF05.loc[dataDF05['현황']=='지연',['현황']].count()
total_late(dataDF06)
totalLateDF['6월'] = dataDF06.loc[dataDF06['현황']=='지연',['현황']].count()
total_late(dataDF07)
totalLateDF['7월'] = dataDF07.loc[dataDF07['현황']=='지연',['현황']].count()
total_late(dataDF08)
totalLateDF['8월'] = dataDF08.loc[dataDF08['현황']=='지연',['현황']].count()
total_late(dataDF09)
totalLateDF['9월'] = dataDF09.loc[dataDF09['현황']=='지연',['현황']].count()
total_late(dataDF10)
totalLateDF['10월'] = dataDF10.loc[dataDF10['현황']=='지연',['현황']].count()
total_late(dataDF11)
totalLateDF['11월'] = dataDF11.loc[dataDF11['현황']=='지연',['현황']].count()
total_late(dataDF12)
totalLateDF['12월'] = dataDF12.loc[dataDF12['현황']=='지연',['현황']].count()


현황    4479
dtype: int64
현황    4474
dtype: int64
현황    4018
dtype: int64
현황    3787
dtype: int64
현황    3940
dtype: int64
현황    5454
dtype: int64
현황    7621
dtype: int64
현황    6324
dtype: int64
현황    5271
dtype: int64
현황    5102
dtype: int64
현황    5515
dtype: int64
현황    6008
dtype: int64


In [171]:
totalLateDF.loc['항공건수'] = [
                    dataDF01.shape[0],
                    dataDF02.shape[0],
                    dataDF03.shape[0],
                    dataDF04.shape[0],
                    dataDF05.shape[0],
                    dataDF06.shape[0],
                    dataDF07.shape[0],
                    dataDF08.shape[0],
                    dataDF09.shape[0],
                    dataDF10.shape[0],
                    dataDF11.shape[0],
                    dataDF12.shape[0]
                    ]

In [172]:
totalLateDF

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
현황,4479,4474,4018,3787,3940,5454,7621,6324,5271,5102,5515,6008
항공건수,17572,16538,17353,16731,17464,17215,18499,18706,17906,18494,17493,18625
